# 경찰청

In [1]:
from bs4 import BeautifulSoup
import pandas as pd 
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import folium
from urllib.parse import quote
import json
import requests

In [2]:
with open('../KakaoAPI.txt') as file:
    kakao_key = file.read()

In [4]:
url = 'https://www.police.go.kr/user/bbs/BD_selectBbsList.do?q_bbsCode=1038&q_tab=1'
driver = webdriver.Chrome()
driver.get(url)

In [5]:
driver.find_element(By.ID, 'tab_1').click()
time.sleep(3)

In [6]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [8]:
lis = soup.select('tbody > tr')
len(lis)

32

In [10]:
li = lis[-1]
li

<tr>
<td class="show-col">서울혜화경찰서</td>
<td class="show-col">서울시 종로구 창경궁로 112-16 </td>
<td class="show-col">
<a href="https://www.smpa.go.kr/hh/" target="_blank" title="새창열림">https://www.smpa.go.kr/hh/</a>
</td>
</tr>

In [12]:
# 경찰서 이름
title = li.select('.show-col')[0].text
title

'서울혜화경찰서'

In [13]:
addr = li.select('.show-col')[1].text
addr

'서울시 종로구 창경궁로 112-16 '

In [14]:
href =  li.select('.show-col')[2].text
href

'\nhttps://www.smpa.go.kr/hh/\n'

In [18]:
lines = []
for li in lis:
   title = li.select('.show-col')[0].text 
   addr = li.select('.show-col')[1].text
   href =  li.select('.show-col')[2].text
   lines.append([title, addr, href])

In [ ]:
lines

In [19]:
df = pd.DataFrame(lines, columns=['관서명', '주소', '홈페이지'])
df.head()

,관서명,주소,홈페이지
0,서울특별시경찰청,서울시 종로구 사직로8길 31,\nhttps://www.smpa.go.kr/\n
1,서울강남경찰서,서울시 강남구 테헤란로 114길 11,\nhttps://www.smpa.go.kr/gn/\n
2,서울강동경찰서,서울시 강동구 성내로 57,\nhttps://www.smpa.go.kr/gd/\n
3,서울강북경찰서,서울시 강북구 오패산로 406,\nhttps://www.smpa.go.kr/gb/\n
4,서울강서경찰서,서울시 강서구 화곡로 308,\nhttps://www.smpa.go.kr/gs/\n


In [21]:
lat_list, lng_list = [], []
for i in df.index:
    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    try:
        lat_list.append(result['documents'][0]['y'])
        lng_list.append(result['documents'][0]['x'])
    except:
        print(df.관서명[i])

서울종암경찰서


In [23]:
# '서울종암경찰서' 주소 확인
df[df.관서명.isin(['서울종암경찰서'])].주소

28    서울시 성북구 종암로 135
Name: 주소, dtype: object

In [ ]:
# 1. 28번행 drop한다
# 2. 새로 검색해서 수정한후 다시 위도경받아오기
서울 성북구 화랑로7길 32

In [24]:
df.iloc[28]

관서명                            서울종암경찰서
주소                     서울시 성북구 종암로 135
홈페이지    \nhttps://www.smpa.go.kr/ca/\n
Name: 28, dtype: object

In [25]:
df.iloc[28]['주소'] = '서울 성북구 화랑로7길 32'

In [26]:
df.iloc[28]

관서명                            서울종암경찰서
주소                     서울 성북구 화랑로7길 32
홈페이지    \nhttps://www.smpa.go.kr/ca/\n
Name: 28, dtype: object

In [ ]:
# 주소 수정해서 위도경도 받아오는지 확인
url = f'{base_url}?query={quote("서울 성북구 화랑로7길 32")}'
result = requests.get(url, headers=header).json()
result

In [28]:
lat_list, lng_list = [], []
for i in df.index:
    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    try:
        lat_list.append(result['documents'][0]['y'])
        lng_list.append(result['documents'][0]['x'])
    except:
        print(df.관서명[i])

In [30]:
df['위도'] = lat_list
df['경도'] = lng_list

In [ ]:
df.tail()

In [ ]:
df.isnull().sum()

# 전국 경찰서 정보 가져오기

In [34]:
# 전국 지역의 수  18개
count = len(soup.select('.nav.nav-tabs.ui-multi-line > li'))
count

18

In [35]:
lines = []
for i in range(1, count + 1):
    driver.find_element(By.ID, f'tab_{i}').click()
    time.sleep(3)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    lis = soup.select('tbody > tr')
    for li in lis:
        title = li.select('.show-col')[0].text 
        addr = li.select('.show-col')[1].text
        href =  li.select('.show-col')[2].text
        lines.append([title, addr, href])

In [ ]:
df = pd.DataFrame(lines, columns=['관서명', '주소', '홈페이지'])
df.head()

In [37]:
len(df)

278

In [38]:
df[df.관서명.isin(['서울종암경찰서'])].주소

28    서울시 성북구 종암로 135
Name: 주소, dtype: object

In [39]:
df.iloc[28]['주소'] = '서울 성북구 화랑로7길 32'

In [ ]:
df.iloc[28]

In [50]:
lat_list, lng_list = [], []
for i in df.index:
    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    try:
        lat_list.append(result['documents'][0]['y'])
        lng_list.append(result['documents'][0]['x'])
    except:
        print(df.관서명[i])
# 전국을 받아오면  123번과 259 번 주소가 잘못된것을 확인 할수 있음

In [ ]:
# 전국을 받아오면 가평경찰서와 밀양경찰서를 받아오지 못함
df[df.관서명.isin(['가평경찰서'])].주소
df[df.관서명.isin(['밀양경찰서'])].주소

In [ ]:
# 카카오 맵에서 검색을 직접하여 주소를 수정
df.iloc[123]['주소'] = '경기 가평군 가평읍 대곡리 229-1'
df.iloc[259]['주소'] = '경남 밀양시 상남면 밀양대로 1545'

In [ ]:
# 수정된 데이터로 다시 위도경도 받아오기
lat_list, lng_list = [], []
for i in df.index:
    base_url = 'https://dapi.kakao.com/v2/local/search/address.json'
    header = {'Authorization': f'KakaoAK {kakao_key}'}
    url = f'{base_url}?query={quote(df["주소"][i])}'
    result = requests.get(url, headers=header).json()
    try:
        lat_list.append(result['documents'][0]['y'])
        lng_list.append(result['documents'][0]['x'])
    except:
        print(df.관서명[i])
# 오류없이 전부 받아온것을 확인후 위도경도 합치기

In [51]:
df['위도'] = lat_list
df['경도'] = lng_list

In [55]:
df.to_csv('전국경찰서.csv', index=False)

In [56]:
map = folium.Map(location=[37.55, 126.98], zoom_start=12)
for i in df.index:
    folium.Marker(
        location=[df.위도[i], df.경도[i]],
        popup=folium.Popup(df.주소[i], max_width=200),
        tooltip=f'{df.관서명[i]}',
        icon=folium.Icon(icon='glyphicon glyphicon-home')
    ).add_to(map)
title_html = '<h3 align="center" style="font-size:20px"> 전국  경찰서 위치</h3>'
map.get_root().html.add_child(folium.Element(title_html))
map